# Quality control of Xenium data

This example is for one sample. Especifically those from 2024-08-15.

__original__ = "2024-11-05 Tue 13:16:29 GMT"

__created__ = "2024-12-03 Tue 21:29:52 GMT"

__updated__ = "2024-12-03"

__version__ = "0.0.9"

__status__ = "Prototype"

__license__ = "GPL"

__maintainer__ = "Ciro Ramírez-Suástegui"

__author__ = "Ciro Ramírez-Suástegui"

__email__ = "cs59@sanger.ac.uk, cramsuig@gmail.com"

### Structure <a class="anchor" id="menu"></a>

* [Global variables and paths](#bullet1)
* [Loading data](#bullet2)
* [Pre-processing](#bullet3)
* [Main](#bullet4)
* [Conclusions](#bullet5)
* [Save](#bullet6)

### Environment setup

### Jupyter extensions

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

### Basic modules

In [ ]:
import os, sys, warnings, re, time
import inspect, pickle

### Logging configuration

In [ ]:
import logging, datetime

In [ ]:
logging.basicConfig(
    format="[%(asctime)s] %(levelname)-2s [%(filename)s:%(lineno)s] %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,  # send to output (no red background)
)
logging.addLevelName(
    logging.INFO, "\033[1;32m%s\033[1;0m" % logging.getLevelName(logging.INFO)
)

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

### Working path

In [ ]:
try:
    __file__
except NameError:
    logging.info("Based on current directory")
    __file__ = os.path.abspath("")
    if os.path.basename(__file__) == "":
        __file__ = os.path.join(os.path.abspath(os.getcwd()), "thisfile.ipynb")

In [ ]:
logging.info(f"Working file: {__file__}")

In [ ]:
project_path = os.popen("git rev-parse --show-toplevel").read().rstrip('\n')
if project_path == "":
    project_path = os.path.dirname(os.path.realpath(__file__))
if os.path.basename(project_path) != os.path.basename(os.getcwd()):
    os.chdir(project_path)

In [ ]:
# show current environment and path
# fmt: off
bpaths = ".*" + os.environ.get('USER', os.environ.get('USERNAME')) + "|.os.py|.*mamba|.*conda"
logging.info(f'Environment: {re.sub(bpaths, "", os.__file__)}')
logging.info(f'Working at: {re.sub(bpaths, "", os.getcwd())}'); del bpaths
logging.info(os.popen("echo Machine: ${HOSTNAME} ${SLURMD_NODENAME} ${PBS_NODEFILE}").read())
# fmt: on

### In-house/developing modules

In [ ]:
sys_path_add = ["code/"]
[sys.path.append(i) for i in sys_path_add if i not in sys.path]
# import filename as shortname

### Tool (packaged) modules

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.figure as mplfig  # to check for Figure type

In [ ]:
import anndata as ad
import scipy.sparse as sp
import seaborn as sns
import squidpy as sq

## [Global variables and paths](#menu) <a class="anchor" id="bullet1"></a>

In [ ]:
sample_path = "20240815__115640__SGP177_SKI_run1/"

In [ ]:
action_name = "qc"
indata_name = re.sub("output-XETG.....__|__........__......$", "", sample_path)
result_name = f"{action_name}_{indata_name}"

In [ ]:
inputs_path = "./data"
inputs_base = os.path.join(
    inputs_path,
    sample_path,
)
inputs_edat = os.path.join(inputs_base, "cell_feature_matrix.h5")
inputs_mdat = os.path.join(inputs_base, "cells.csv.gz")

In [ ]:
output_resu = os.path.join("./results", f"{result_name}")
output_figs = os.path.join("./figures", f"{result_name}")
output_name = indata_name
output_file = os.path.join(inputs_path, f"{output_name}_{action_name}.h5ad")

In [ ]:
OUTPUTS = dict()

In [ ]:
%whos str dict

### Visualisation parameters

In [ ]:
rcParams_dict = {
    "figure.autolayout": True, # use `tight_layout`
    "figure.dpi": 80,  # scanpy:80
    "figure.figsize": (4, 4),  # scanpy:4x4
    "figure.frameon": False,
    "grid.linestyle": "dotted",
    "grid.color": "#f2f2f2",
    "font.size": 10,  # scanpy:14
    # https://matplotlib.org/stable/users/explain/colors/colormaps.html
    "image.cmap": "viridis",
    "lines.markersize": 2,  # dotplot size
    "savefig.dpi": 150,  # default is 'figure', scanpy:150
    "savefig.bbox": "tight",
    "savefig.transparent": True,
}

### Scanpy settings and logging

In [ ]:
sc.logging.print_versions()

In [ ]:
sc.settings.set_figure_params(dpi_save=rcParams_dict["savefig.dpi"])
sc.settings.set_figure_params(**{
    re.sub("figure|\.", "", k): rcParams_dict[k]
    for k in rcParams_dict.keys()
    if re.match("figure|font", k) and re.match("^((?!layout).)*$", k)
})
sc.settings.figdir = output_figs

In [ ]:
# plt.style.available or path (.config/rparams_analysis.mplstyle)
plt.style.use("seaborn-v0_8-colorblind")
rcmax = max([len(i) for i in list(rcParams_dict.keys())])
for i in rcParams_dict.keys():
    i_def = "def:" + str(plt.rcParamsDefault[i]).rjust(10, " ")
    i_new = str(plt.rcParams[i]).rjust(10, " ")
    temp = " ".join([i.ljust(rcmax, " "), i_def, "|", i_new, ">"])
    plt.rcParams[i] = rcParams_dict[i]
    temp = " ".join([temp, str(plt.rcParams[i])])
    logging.info(temp)

## [Loading data](#menu) <a class="anchor" id="bullet2"></a>

In [ ]:
adata = sc.read_10x_h5(inputs_edat)

In [ ]:
adata

In [ ]:
metadata = pd.read_csv(inputs_mdat)

In [ ]:
metadata

## [Pre-processing](#menu) <a class="anchor" id="bullet3"></a>

In [ ]:
metadata.set_index(adata.obs_names, inplace=True)
adata.obs = metadata.copy()

In [ ]:
adata.obs.head()

In [ ]:
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

## [Main](#menu) <a class="anchor" id="bullet4"></a>

### Perform basic QC analysis á la Squidpy

In [ ]:
%%time
sc.pp.calculate_qc_metrics(adata, percent_top = (10, 20, 50, 150), inplace = True)

Calculate percentage of control probes and control codewords

In [ ]:
cprobes = (
    adata.obs["control_probe_counts"].sum() / adata.obs["total_counts"].sum() * 100
)
cwords = (
    adata.obs["control_codeword_counts"].sum() / adata.obs["total_counts"].sum() * 100
)
print(f"Negative DNA probe count % : {cprobes:.4f}")
print(f"Negative decoding count % : {cwords:.4f}")

Plot distribution of
* Total transcripts per cell
* Unique transcripts per cell
* Area of segmented cells
* Ratio of nuclei area to their cells

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(16, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    adata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Number of genes detected per cell")
sns.histplot(
    adata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)


axs[2].set_title("Area of segmented cells (μm^2)")
sns.histplot(
    adata.obs["cell_area"],
    kde=False,
    ax=axs[2],
)

axs[3].set_title("Nucleus ratio")
sns.histplot(
    adata.obs["nucleus_area"] / adata.obs["cell_area"],
    kde=False,
    ax=axs[3],
)
OUTPUTS["qc-metrics_hist"] = fig

In [ ]:
# Plot distribution of total cells per gene per segmentation method
fig, axs = plt.subplots(1, 2, figsize=(15, 4))
sns.countplot(
    data=adata.obs, y="segmentation_method", hue="segmentation_method", ax=axs[0]
)
axs[1].set_title("Number of cells per gene")
sns.histplot(
    adata.var["n_cells_by_counts"], kde=False, bins=1000, ax=axs[1]
);  # fmt: skip
OUTPUTS["segmentation_hist"] = fig

In [ ]:
OUTPUTS["spatial_total-counts_scatter"] = sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color="total_counts",
    return_ax=True,
).get_figure()

In [ ]:
OUTPUTS["spatial_n-genes-by-counts_scatter"] = sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color="n_genes_by_counts",
    return_ax=True,
).get_figure()

### Necrosis

Low * content might indicate necrotic tissue, therefore, we would like to remove
at the dip in a biomodal distribution.

In [ ]:
# Change the upper bound of the range to where you think makes sense.
ax = plt.hist(adata.obs["n_genes_by_counts"], bins=50, range=(0, 500))
fig = plt.gcf()
n, bins = ax[0], ax[1]
min_index = n.argmin()
necrosis_cutoff_genes = (bins[min_index] + bins[min_index + 1]) / 2
plt.axvline(x=necrosis_cutoff_genes, color="#800f0f");  # fmt: skip
OUTPUTS["necrosis-cutoff_n-genes-by-counts"] = fig

In [ ]:
ax = plt.hist(adata.obs["total_counts"], bins=50, range=(0, 500))
fig = plt.gcf()
n, bins = ax[0], ax[1]
min_index = n.argmin()
necrosis_cutoff_transcripts = (bins[min_index] + bins[min_index + 1]) / 2
plt.axvline(x=necrosis_cutoff_transcripts, color="#800f0f");  # fmt: skip
OUTPUTS["necrosis-cutoff_total-counts"] = fig

In [ ]:
adata.obs["necrotic"] = (adata.obs["n_genes_by_counts"] < necrosis_cutoff_genes) | (
    adata.obs["total_counts"] < necrosis_cutoff_transcripts
)

Check the spatial distribution of the tissue label as necrotic.

In [ ]:
OUTPUTS["necrosis_spatial"] = sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color="necrotic",
    return_ax=True,
).get_figure()

### Filter cells

In [ ]:
sc.pp.filter_cells(adata, min_genes=3)

In [ ]:
sc.pp.filter_genes(adata, min_counts=10)

In [ ]:
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
adata

### Processing counts

In [ ]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum=10000, inplace=True)

In [ ]:
sc.pp.log1p(adata)

### Clustering

This usually is given from a reference (suspension data) but we can identify larger structructures, including the necrotic region, by clustering here.

In [ ]:
%%time
sc.pp.pca(adata)

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
%%time
sc.pp.neighbors(adata, n_pcs=25)

In [ ]:
%%time
sc.tl.leiden(adata, resolution = 0.5)

In [ ]:
%%time
sc.tl.umap(adata)

### Composition

In [ ]:
OUTPUTS["leiden_umap"] = sc.pl.umap(
    adata,
    color=[
        "total_counts",
        "n_genes_by_counts",
        "leiden",
    ],
    return_fig=True,
)

In [ ]:
OUTPUTS["leiden_spatial"] = sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,  # fails if not None
    color="leiden",
    return_ax=True,
).get_figure()

Checking which is enriched for necrotic tissue.

In [ ]:
fig, axs = plt.subplots()
sns.histplot(adata[adata.obs["necrotic"] == True].obs["leiden"], ax=axs)
OUTPUTS["necrosis_leiden_hist"] = fig

### Markers

We just need to make sure we are not discarding important cells in the necrotic
region.

In [ ]:
%%time
sc.tl.rank_genes_groups(adata, 'leiden', method = 'wilcoxon', n_genes = 100, use_raw = False)

In [ ]:
OUTPUTS["features_leiden_dotplot_top5"] = sc.pl.rank_genes_groups_dotplot(
    adata, groupby="leiden", standard_scale="var", n_genes=5, return_fig=True
)

## [Conclusions](#menu) <a class="anchor" id="bullet5"></a>

In [ ]:
temp = list(OUTPUTS.keys())
temp

In [ ]:
for i in temp:
    if len(re.findall(r"params|colours", i)) > 0:
        del OUTPUTS[i]

In [ ]:
OUTPUTS.keys()

## [Save](#menu) <a class="anchor" id="bullet6"></a>

In [ ]:
for i in [output_figs, output_resu]:
    if not os.path.isdir(i):
        os.makedirs(i)

In [ ]:
pflag = " \033[1;32m√\033[0m"
cflag = 0
for filename, item in OUTPUTS.items():
    if isinstance(item, (pd.DataFrame, dict)):
        fname = os.path.join(output_resu, f"{str(cflag).zfill(2)}_{filename}")
    else:
        fname = os.path.join(output_figs, f"{str(cflag).zfill(2)}_{filename}")
    print("Storing", type(item), "\n", fname, end="")
    eflag = " \033[1;31mX\033[0m"
    if not os.path.isdir(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        eflag = pflag
    # file type # ------------------------------------------
    if isinstance(item, (mplfig.Figure, tuple)):
        (item[0] if isinstance(item, tuple) else item).savefig(f"{fname}.png")
        plt.close()
        eflag = pflag
    elif isinstance(item, pd.DataFrame):
        item.to_csv(f"{fname}.csv")
        eflag = pflag
    elif isinstance(item, dict):
        with open(f"{fname}.pickle", "wb") as handle:
            pickle.dump(item, handle, protocol=pickle.HIGHEST_PROTOCOL)
        eflag = pflag
    elif item is not None:
        item.savefig(f"{fname}.png")
        plt.close()
        eflag = pflag
    print(f"{eflag}")
    cflag += 1

Done.